In [2]:
import os 
import glob 

import numpy as np
import pandas as pd

from metpy.calc import wind_speed
from metpy.calc import wind_direction 
from metpy.units import units

from scipy import interpolate

import matplotlib.pyplot as plt
from   matplotlib.lines import Line2D
import seaborn as sns

sns.set_theme(style="whitegrid")

# About
__Author:__ Pat McCornack

__Date:__ 12/26/24

__Description:__   
This is a tool to bias correct the WRF data to match the distribution of the observational data for a given set of variables.

__Output:__  
Saves out a dataframe for each site containing the bias corrected WRF data. 

__To Do:__  
1. If revisited, convert this to a command line tool. 

# Functions

## Calculations

In [3]:
def get_wind(u, v): 
    """"
    Calculates wind speed and direction from velocity components

    Parameters
    -----------
    u : np.Series
        The x wind component.
    v : np.Series  
        The y wind component. 

    Returns
    -------
    wind_speed : float
        Calculated wind speed.
    wind_dir : float
        Calculated wind direction.
    """
    u = u * units('m/s')
    v = v * units('m/s')

    wnd_spd = wind_speed(u,v)
    wnd_dir = wind_direction(u, v, convention='from')
    
    return wnd_spd, wnd_dir

def quantile_map(wrf_series, obs_series):
    """
    Performs quantile mapping to correct bias in the WRF data.

    Parameters
    ----------
    wrf_series : pd.Series
        Series of data for a given variable extracted from the WRF output (e.g. temperature).
    obs_series: pd.Series
        Series of data for the same variable from the observational dataset.
        
    Returns
    -------
    pd.Series
        The WRF series corrected using quantile mapping.


    """
    quantiles = np.arange(.1, 1, .1)

    obs_quantiles = obs_series.quantile(quantiles)
    wrf_quantiles = wrf_series.quantile(quantiles)
    
    correction_function = interpolate.interp1d(wrf_quantiles, obs_quantiles,
                                           bounds_error=False, fill_value="extrapolate")
    corrected = correction_function(wrf_series)
    
    return corrected

## Read in Data 

In [4]:
def prep_wrf_data(fpath, var_name_dict, site):
    """
    Reads and prepares WRF data

    Reads in data, convert temperature from K to C, casts index to DateTime format, calculates
    wind speed and direction, and returns the prepared dataframe

    Parameters
    ----------
    df_fpath : str
        Path to file containing the data.
    var_name_dict : dict
        Dictionary of variable rename mappings to.
    
    Returns
    -------
    pd.Dataframe
        The prepared dataframe. 
    """
    df = pd.read_pickle(fpath)
    df['T2'] = df['T2'] - 273  # Convert K to C
    df.index = df.index - pd.Timedelta(hours=8)  # UTC to local
    wnd_spd, wnd_dir = get_wind(df['U10'].values, df['V10'].values)
    df['WND_SPD'] = wnd_spd
    df['WND_DIR'] = wnd_dir
    df.rename(columns=var_name_dict, inplace=True)
    
    df.attrs['site'] = site
    df.attrs['source'] = 'wrf'
    
    return df

def prep_obs_data(fpath, site):
    """
    Reads and prepares WRF data

    Reads in data, convert temperature from K to C, casts index to DateTime format, calculates
    wind speed and direction, and returns the prepared dataframe

    Parameters
    ----------
    fpath : str
        Path to file containing the data.
    site : str 
        The site the data corresponds to.

    Returns
    -------
    pd.Dataframe
        The prepared dataframe. 
    """
    df = pd.read_csv(fpath, index_col='time (PST)')
    df.index = pd.to_datetime(df.index)
    
    df.attrs['site'] = site
    df.attrs['source'] = 'obs'
    
    return df

# Set up working directory - adjust this to where the repo is stored locally
%cd '/Users/patmccornack/Documents/ucsb_fog_project/_repositories/sci-wrf-analysis'

/Users/patmccornack/Documents/ucsb_fog_project/_repositories/sci-wrf-analysis


# Read in and Prepare Data

In [7]:
var_name_dict = {
    'DFGDP' : 'fog drip',
    'T2' : 'air temperature (C)',
    'RH' : 'relative humidity (%)',
    'WND_SPD' : 'wind speed (m/s)',
    'WND_DIR' : 'wind direction (deg)'
}

#### WRF file paths ####
wdatadir = './data/wrf-extracted/single-pixel'
wsauc_fname = 'wrf-sauc-2003-2010.pkl'
wupem_fname = 'wrf-upem-2003-2010.pkl'
wnrs_sci_fname = 'wrf-nrs-2014-2019.pkl'

#### Obs file paths ####
odatadir = './data/observational'
osauc_fname = 'sauc-hourly.csv'
oupem_fname = 'upem-hourly.csv'
onrs_sci_fname = 'nrs-hourly.csv'

#### Read in data ####
wsauc = prep_wrf_data(os.path.join(wdatadir, wsauc_fname), var_name_dict, site='Sauces Canyon')
wupem = prep_wrf_data(os.path.join(wdatadir, wupem_fname), var_name_dict, site='Upper Embudo Canyon')
wnrs_sci = prep_wrf_data(os.path.join(wdatadir, wnrs_sci_fname), var_name_dict, site='NRS SCI')

osauc = prep_obs_data(os.path.join(odatadir, osauc_fname), site='Sauces Canyon')
oupem = prep_obs_data(os.path.join(odatadir, oupem_fname), site='Upper Embudo Canyon')
onrs_sci = prep_obs_data(os.path.join(odatadir, onrs_sci_fname), site='NRS SCI')

In [12]:
# Join all into a single dataframe
df_list = [wsauc, wupem, wnrs_sci, osauc, oupem, onrs_sci]
for df in df_list:
    df['site'] = df.attrs['site']
    df['source'] = df.attrs['source']

df = pd.concat(df_list, join='inner', axis=0)
df.tail(2)

,air temperature (C),relative humidity (%),wind speed (m/s),wind direction (deg),site,source
2019-12-31 22:00:00,13.213333,51.555000,2.159500,19.641667,NRS SCI,obs
2019-12-31 23:00:00,13.298333,52.808333,2.273167,78.286667,NRS SCI,obs


# Define output file paths

In [13]:
outdir = "~/data/wrf-extracted/quantile-mapped"
pd.to_pickle(df.loc[df['source']=='wrf'], os.path.join(outdir, "wrf-qm-nrs-sauc-upem.pkl"))

# Perform quantile mapping
Correct the WRF data distribution for each variable in var_list using quantile mapping.

In [7]:
var_list = ['air temperature (C)', 'relative humidity (%)', 'wind speed (m/s)', 'wind direction (deg)']

In [8]:
# Map
qmwsauc = wsauc[var_list].copy()
for var in var_list:
    qmwsauc[var] = quantile_map(qmwsauc[var], osauc[var])

qmwupem = wupem[var_list].copy()
for var in var_list:
    qmwupem[var] = quantile_map(qmwupem[var], oupem[var])
    
qmwnrs_sci = wnrs_sci[var_list].copy()
for var in var_list:
    qmwnrs_sci[var] = quantile_map(qmwnrs_sci[var], onrs_sci[var])
    
    
# Save out
qmwsauc.to_pickle(os.path.join(outdir, wsauc_fname))
qmwupem.to_pickle(os.path.join(outdir, wupem_fname))
qmwnrs_sci.to_pickle(os.path.join(outdir, wnrs_sci_fname))